In [ ]:
from systems.basesystem import System
from systems.positionsizing import PositionSizing
from systems.forecast_combine import ForecastCombine
from systems.forecast_scale_cap import ForecastScaleCap
from systems.trading_rules import TradingRule
from systems.forecasting import Rules
from systems.accounts.accounts_stage import Account
from systems.portfolio import Portfolios
from systems.rawdata import RawData
from systems.provided.rules.ewmac import ewmac, ewmac_calc_vol
from systems.provided.rules.breakout import breakout
from systems.provided.rules.accel import accel
from sysdata.sim.db_equities_sim_data import dbEquitiesSimData
from sysdata.config.configdata import Config

import matplotlib.pyplot as plt
import pandas as pd

import math


In [ ]:
def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    inv = ax2.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, y1-y2))
    # miny, maxy = ax1.get_ylim()
    # ax1.set_ylim(miny+dy, maxy+dy)
    miny, maxy = ax2.get_ylim()
    ax2.set_ylim(miny+dy, maxy+dy)


def align_yaxis(ax1, ax2):
    """Align zeros of the two axes, zooming them out by same ratio"""
    axes = (ax1, ax2)
    extrema = [ax.get_ylim() for ax in axes]
    tops = [extr[1] / (extr[1] - extr[0]) for extr in extrema]
    # Ensure that plots (intervals) are ordered bottom to top:
    if tops[0] > tops[1]:
        axes, extrema, tops = [list(reversed(l)) for l in (axes, extrema, tops)]

    # How much would the plot overflow if we kept current zoom levels?
    tot_span = tops[1] + 1 - tops[0]

    b_new_t = extrema[0][0] + tot_span * (extrema[0][1] - extrema[0][0])
    t_new_b = extrema[1][1] - tot_span * (extrema[1][1] - extrema[1][0])
    axes[0].set_ylim(extrema[0][0], b_new_t)
    axes[1].set_ylim(t_new_b, extrema[1][1])


In [ ]:
class MyPositionSizing(PositionSizing):

    def __init__(self, *args, long_only=False, **kwargs):
        self.__long_only = long_only
        super().__init__(*args, **kwargs)

    def get_subsystem_position(self, instrument_code: str) -> pd.Series:
        subsystem_position = super().get_subsystem_position(instrument_code)
        if self.__long_only:
            subsystem_position[subsystem_position < 0] = 0
        return subsystem_position

In [ ]:
from datetime import datetime, timedelta
from functools import cached_property
from gzip import FEXTRA

years = 2
start_date = str((datetime.now()-timedelta(days=years*365)).date())

rules = Rules(dict(
    mac4 = TradingRule(ewmac, ['rawdata.get_daily_prices', 'rawdata.daily_returns_volatility'], {'Lfast':4, 'Lslow':16}),
    mac8 = TradingRule(ewmac, ['rawdata.get_daily_prices', 'rawdata.daily_returns_volatility'], {'Lfast':8, 'Lslow':32}),
    mac16 = TradingRule(ewmac, ['rawdata.get_daily_prices', 'rawdata.daily_returns_volatility'], {'Lfast':16, 'Lslow':64}),
    mac32 = TradingRule(ewmac, ['rawdata.get_daily_prices', 'rawdata.daily_returns_volatility'], {'Lfast':32, 'Lslow':128}),
    mac64 = TradingRule(ewmac, ['rawdata.get_daily_prices', 'rawdata.daily_returns_volatility'], {'Lfast':64, 'Lslow':256}),

    normmom2 = TradingRule(ewmac_calc_vol, ['rawdata.get_cumulative_daily_vol_normalised_returns'], {'Lfast':2, 'Lslow':8}),
    normmom4 = TradingRule(ewmac_calc_vol, ['rawdata.get_cumulative_daily_vol_normalised_returns'], {'Lfast':4, 'Lslow':16}),
    normmom8 = TradingRule(ewmac_calc_vol, ['rawdata.get_cumulative_daily_vol_normalised_returns'], {'Lfast':8, 'Lslow':32}),
    normmom16 = TradingRule(ewmac_calc_vol, ['rawdata.get_cumulative_daily_vol_normalised_returns'], {'Lfast':16, 'Lslow':64}),
    normmom32 = TradingRule(ewmac_calc_vol, ['rawdata.get_cumulative_daily_vol_normalised_returns'], {'Lfast':32, 'Lslow':128}),
    normmom64 = TradingRule(ewmac_calc_vol, ['rawdata.get_cumulative_daily_vol_normalised_returns'], {'Lfast':64, 'Lslow':256}),
    
    breakout10 = TradingRule(breakout, [], {'lookback':10}),
    breakout20 = TradingRule(breakout, [], {'lookback':20}),
    breakout40 = TradingRule(breakout, [], {'lookback':40}),
    breakout80 = TradingRule(breakout, [], {'lookback':80}),
    breakout160 = TradingRule(breakout, [], {'lookback':160}),
    breakout320 = TradingRule(breakout, [], {'lookback':320}),

    accel16 = TradingRule(accel, ['rawdata.get_daily_prices', 'rawdata.daily_returns_volatility'], {'Lfast':16}),
    accel32 = TradingRule(accel, ['rawdata.get_daily_prices', 'rawdata.daily_returns_volatility'], {'Lfast':32}),
    accel64 = TradingRule(accel, ['rawdata.get_daily_prices', 'rawdata.daily_returns_volatility'], {'Lfast':64}),
))

# # Dragon
# instrument_weights = dict(
#     SPY = 0.2,
#     AGG = 0.2,
#     GLD = 0.2,
#     DJP = 0.2,
#     QAI = 0.2,
# )


instrument_weights = dict(
    SPY = 0.25,
    QQQ = 0.25,
    VYM = 0.25,
    EEM = 0.25,

    AGG = 0.2,
    IEF = 0.2,
    TLT = 0.2,
    LQD = 0.2,
    HYG = 0.2,

    GLD = 0.25,
    BITO = 0.25,
    UUP = 0.25,
    FXE = 0.25,

    PDBC = 0.25,
    USO = 0.125,
    UNG = 0.125,
    DBB = 0.25,
    DBA = 0.25,
)



# instrument_weights = dict(
#     IWD = 0.33,
#     IWF = 0.33,
#     # EEM = 0.33,

#     TLT = 0.25,
#     IEF = 0.25,
#     HYG = 0.25,
#     LQD = 0.25,
    
#     GLD = 0.5,
#     BITO = 0.5,
    
#     PDBC = 1,
# )

# # Commodities
# instrument_weights = dict(
#     **instrument_weights, 
#     USO     = 0.2,
#     URNM    = 0.2, 
#     KRBN    = 0.2,
#     LIT     = 0.2,
#     CANE    = 0.2,
#     COW     = 0.2,
# )

# # International
# instrument_weights = dict(
#     **instrument_weights, 
#     MCHI = 0.2,
#     INDA = 0.2,
#     EWT = 0.2,
#     EWY = 0.2,
#     EWZ = 0.2,
#     # KSA = 0.2,
#     # EWW = 0.2,
#     # EIDO = 0.2,
#     # TUR = 0.2
# )

# # Currencies
# instrument_weights = dict(
#     **instrument_weights,
#     # UUP     = 0.2,
#     FXB     = 0.2,
#     FXE     = 0.2,
#     # FXY     = 0.2,
#     # FXC     = 0.2,
# )

# # Vincent Deluard - 2022-11-04
# instrument_weights = dict(
#     **instrument_weights,
#     XLF = 0.33,
#     XLV = 0.33,
#     XLE = 0.33,
# )

# instrument_weights = dict(
#     **instrument_weights, 
#     PFIX = 0.2,
# )

instruments = list(instrument_weights.keys())

config = Config(['private.estimates.yaml'])

class MySystem:

    def __init__(self, name, long_only=False):
        system = System([rules, RawData(), ForecastScaleCap(), ForecastCombine(), MyPositionSizing(long_only=long_only), Account(), Portfolios()], data=dbEquitiesSimData(), config=config)
        system.config.start_date = start_date
        system.config.notional_trading_capital = 100000
        system.config.percentage_vol_target = 25
        system.config.instruments = list(instrument_weights.keys())
        # system.config.instrument_weights = instrument_weights
        system.config.instrument_div_multiplier = 2.0
        # system.config.use_instrument_div_mult_estimates = True
        # system.config.use_instrument_weight_estimates = True
        # system.config.use_forecast_scale_estimates = True
        # system.config.use_forecast_div_mult_estimates = True
        # system.config.use_forecast_weight_estimates = True
        system.config.capital_multiplier['func'] = 'syscore.capital.full_compounding'
        # system.config.buffer_trade_to_edge = False
        # system.config.buffer_method = 'position'
        # system.config.buffer_size = 0.1
        print(system.get_instrument_list())
        self.system = system
        self.name = name


    def plot_combined_forecasts(self):


        n_instruments = len(instruments)
        n_cols = 6
        n_rows = math.ceil(n_instruments/n_cols)
        dims = (n_rows, n_cols)

        fig, ax = plt.subplots(*dims)
        for i, instrument in enumerate(sorted(instruments)):
            
            axis = ax[i//n_cols][i%n_cols] if n_rows > 1 else ax[i%n_cols]
            axis.set_title(instrument)
            self.system.combForecast.get_combined_forecast(instrument).plot(ax=axis, lw=3)

        plt.tight_layout()
        plt.show()

    def plot_account_curves(self):
        portfolio = self.system.accounts.portfolio_with_multiplier()
        fig, ax1 = plt.subplots()
        portfolio.percent.curve().plot(ax=ax1, color='orange')
        portfolio.percent.drawdown().plot(ax=ax1, color='red')
        ax2 = ax1.twinx()
        portfolio.curve().plot(ax=ax2)

        align_yaxis(ax1,ax2)
        # ax1.set_yticks(step=25)
        ax1.minorticks_on()
        plt.show()

    def display_stats(self):
        display(self.system.accounts.portfolio().percent.stats())


    def get_weights(self):

        if getattr(self, 'weights', None) is not None:
            return self.weights

        def get_percent_portfolio_weight(instrument):
            return self.system.accounts.get_buffered_position_with_multiplier(instrument)*self.system.data.daily_prices(instrument)/self.system.accounts.get_actual_capital()

        weights = pd.DataFrame()
        for i in self.system.get_instrument_list():
            weights[i] = get_percent_portfolio_weight(i)

        self.weights = weights.fillna(method='ffill')

        return self.weights


    def reduce_leverage(self, weights, max_leverage):
        weights = weights.copy()
        total_weight = weights.abs().sum(axis=1)
        weights.loc[total_weight > max_leverage] = weights.mul(0.95).T.div(total_weight).T
        return weights


    def get_buffered_weights(self, buffer=0.1):

        if getattr(self, 'buffered_weights', None) is not None:
            return self.buffered_weights

        weights = self.get_weights()

        w = weights.iloc[0]
        buffered_weights = [w]
        for i in range(1, len(weights)):
            w1 = weights.iloc[i]
            w = w1.mask((w1-w).abs() < (buffer*w).abs(), w)
            buffered_weights.append(w)

        self.buffered_weights = pd.concat(buffered_weights, axis=1).T

        return self.buffered_weights

    def display_weights(self):
        display(self.get_weights())

    def write_ibkr_rebalance_file(self, max_leverage=None):
        from sysbrokers.IB.utils import write_ibkr_rebalance_file
        weights = self.get_weights()

        if max_leverage is not None:
            weights = self.reduce_leverage(weights, max_leverage)

        current_weights = {k:v for k,v in zip(weights.columns, weights[-1:].values[0])}
        current_weights

        write_ibkr_rebalance_file(self.name, current_weights, leverage=1)



In [ ]:
mySystem = MySystem('trend_following_long_short_v1')


In [ ]:
plt.rc('figure', figsize=(30, 15))
mySystem.plot_combined_forecasts()
mySystem.plot_account_curves()
mySystem.display_stats()
mySystem.display_weights()
mySystem.write_ibkr_rebalance_file()

In [ ]:
# symbol = 'QAI'

# positions = mySystem.system.accounts.get_buffered_position_with_multiplier(symbol).rename('positions')
# weights = mySystem.get_weights()[symbol].rename('weights')
# df = pd.concat([positions, weights], axis=1)
# fig, ax1 = plt.subplots()
# ax1.plot(positions)
# ax2 = ax1.twinx()
# ax2.spines['right'].set_color('red')
# ax2.plot(weights, c='red')
# ax3 = ax1.twinx()
# ax3.spines['right'].set_position(('outward', 100))
# ax3.spines['right'].set_color('orange')
# ax3.plot(mySystem.system.combForecast.get_combined_forecast(symbol), c='orange')


In [ ]:
mySystem = MySystem('trend_following_long_only_v1', long_only=True)


In [ ]:
plt.rc('figure', figsize=(30, 15))
mySystem.plot_combined_forecasts()
mySystem.plot_account_curves()
mySystem.display_stats()
mySystem.display_weights()
mySystem.write_ibkr_rebalance_file(max_leverage=0.95)

In [ ]:
from systems.diagoutput import systemDiag
from syscore.fileutils import get_filename_for_package

diag = systemDiag(mySystem.system)
display(diag.output_config_with_estimated_parameters())

def save_estimated_system(system, package):
    filename=get_filename_for_package(package)
    sysdiag = systemDiag(system)
    sysdiag.yaml_config_with_estimated_parameters(filename,
                                                attr_names=['forecast_scalars',
                                                            'forecast_weights',
                                                            'forecast_div_multiplier',
                                                            'forecast_mapping',
                                                            'instrument_weights',
                                                            'instrument_div_multiplier'])

save_estimated_system(mySystem.system, 'private.estimates-test.yaml')